# Visão Computacional - Projeto 2-1

### Bruna Kimura

In [2]:
%matplotlib inline

import cv2 as cv2
import numpy as np     
import matplotlib.pyplot as plt

import time

### 1. Detecção de alguns pontos interessantes na imagem

In [ ]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    # Converte para Grayscale
    bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Parametriza a funcao do OpenCV
    params = dict( maxCorners = 100,
                   qualityLevel = 0.3,
                   minDistance = 7,
                   blockSize = 7 )

    # Funcao que retorna uma lista de pontos
    pts = cv2.goodFeaturesToTrack(bw, mask = None, **params)
    
    # Gera cores de forma aleatória
    color = np.random.randint(0,255,(100,3))

    # Insere uma marcação em cada ponto:
    for i,pt in enumerate(pts):
        a,b = pt.ravel()
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)

    cv2.imshow("Video", frame)

captura.release()
cv2.destroyAllWindows()

### 2. Rastreamento dos pontos (apenas em dois frames)

In [5]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

ret, frame = captura.read()
previous = frame
previous_gray = cv2.cvtColor(previous, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)

# Cria uma máscara para imprimir o rastro.
mask = np.zeros_like(previous)


while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    actual = frame
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_BGR2GRAY)
    
    # Calcula o Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
    
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    
    
    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
        
    
    # diferença entre os pontos antigos e os atuais
    dif = np.subtract(good_old, good_new)
    print(dif)
    
    #a primeira coluna são os x's e a segunda o y', precisa tirar as médias p saber o quanto precisa fazer resize
    #resize da imagem
       
    
    img = cv2.add(actual, mask)
    
    cv2.imshow("Video", img)
    
    # Now update the previous frame and previous points
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    
captura.release()
cv2.destroyAllWindows()


[[ 0.0032959   0.2965393 ]
 [-0.01159668  0.2548523 ]
 [-0.02832031  0.3562622 ]
 [-0.03759766  0.34057617]
 [ 0.02056885  0.44595337]
 [-0.04492188  0.31765747]
 [ 0.02584839  0.35092163]
 [ 0.00085449  0.31210327]
 [-0.00079346  0.2690735 ]
 [-0.09093094  0.34048462]
 [ 0.01776123  0.27459717]
 [ 0.0480957   0.25897217]
 [-0.01434326  0.30737305]
 [-0.15252686  0.3614502 ]
 [-0.0256958   0.30230713]]
[[ 0.05456543 -0.0032959 ]
 [ 0.01141357  0.00552368]
 [ 0.00982666 -0.0088501 ]
 [ 0.04681396  0.01132202]
 [ 0.0118103  -0.01248169]
 [-0.02764893 -0.02862549]
 [-0.04197693  0.10678101]
 [ 0.02182007  0.021698  ]
 [ 0.03979492 -0.01739502]
 [ 0.04146194 -0.10568237]
 [ 0.01751709 -0.0524292 ]
 [ 0.05438232 -0.05215454]
 [ 0.03460693  0.00534058]
 [-0.14259338  0.00485229]
 [ 0.03302002 -0.0697937 ]]
[[-0.01757812  0.00708008]
 [ 0.00750732 -0.0039978 ]
 [ 0.00668335 -0.00991821]
 [ 0.05004883 -0.03424072]
 [ 0.01287842 -0.00961304]
 [ 0.0010376   0.01547241]
 [ 0.0039978  -0.12469482]

[[ 0.03106689 -0.24783325]
 [ 0.0645752  -0.19186401]
 [ 0.00787354 -0.20684814]
 [-0.02539062 -0.24682617]
 [ 0.14590454 -0.5767517 ]
 [-0.01422119 -0.26535034]
 [ 0.00553894 -0.28248596]
 [ 0.9315796  -0.28414917]
 [ 0.04986572 -0.24880981]
 [ 0.07715607 -0.2584839 ]
 [ 0.09088135 -0.23669434]
 [-0.01477051 -0.22949219]
 [-0.0201416  -0.26113892]
 [ 0.7535858  -0.27218628]
 [ 0.15551758 -0.20910645]]
[[ 0.01251221 -0.44302368]
 [-0.03521729 -0.44796753]
 [ 0.03292847 -0.43173218]
 [ 0.05688477 -0.41973877]
 [ 0.35308838 -0.99902344]
 [-0.05194092 -0.42907715]
 [ 0.00445557 -0.43623352]
 [ 1.6462402  -0.5645447 ]
 [-0.00549316 -0.45236206]
 [ 0.11437225 -0.45343018]
 [-0.03283691 -0.48550415]
 [ 0.00689697 -0.469635  ]
 [ 0.06848145 -0.4269104 ]
 [ 1.3041382  -0.41177368]
 [-0.13256836 -0.39538574]]
[[ 0.03222656 -0.177948  ]
 [ 0.00628662 -0.23294067]
 [ 0.00198364 -0.22039795]
 [ 0.01116943 -0.24090576]
 [ 0.04418945 -0.29971313]
 [ 0.04077148 -0.20883179]
 [ 0.00710297 -0.06074524]

### Teste
